# Week 6: BERT Fine-tuning Lab - Enhanced Edition
## Understanding Pre-trained Models Through Hands-on Practice

### 🚀 The Paradigm Shift in NLP

In this enhanced notebook, we'll explore:
1. **The Computational Revolution** - Why pre-training changed everything
2. **Masked Language Modeling** - How BERT learns from text
3. **Fine-tuning vs Training from Scratch** - See the 100x difference
4. **LoRA and Efficient Fine-tuning** - Modern techniques
5. **Environmental Impact** - Calculate carbon footprint
6. **Attention Visualization** - See what BERT "sees"

**Time to complete**: 60 minutes
**Prerequisites**: Basic Python, PyTorch basics
**New**: Interactive paradigm shift demonstrations!

## Part 1: Setup and Installation

First, let's install the necessary libraries. This might take a few minutes.

In [ ]:
# Install required packages (run once)
!pip install transformers torch datasets matplotlib seaborn numpy pandas scikit-learn -q

print("Installation complete!")

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    BertTokenizer, 
    BertForMaskedLM,
    BertForSequenceClassification,
    BertConfig,
    pipeline,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# The Paradigm Shift: Computational Cost Calculator
# This demonstrates why pre-training revolutionized NLP

class ParadigmShiftCalculator:
    """Calculate and visualize the impact of pre-training"""
    
    def __init__(self):
        self.cost_per_model = 500_000  # USD
        self.training_time = 336  # hours (2 weeks)
        self.gpu_cost_per_hour = 3  # USD
        self.carbon_per_kwh = 0.5  # kg CO2
        self.kwh_per_gpu_hour = 0.4  # kilowatt hours
    
    def calculate_traditional_approach(self, n_companies=20):
        """Calculate cost for traditional approach (each company trains from scratch)"""
        total_cost = n_companies * self.cost_per_model
        total_time = n_companies * self.training_time
        total_gpu_cost = total_time * self.gpu_cost_per_hour
        
        # 90% of effort is duplicate (learning general language)
        duplicate_cost = total_cost * 0.9
        
        # Carbon footprint
        total_carbon = n_companies * self.training_time * self.kwh_per_gpu_hour * self.carbon_per_kwh
        
        return {
            'total_cost': total_cost,
            'duplicate_cost': duplicate_cost,
            'total_time': total_time,
            'gpu_cost': total_gpu_cost,
            'carbon_kg': total_carbon
        }
    
    def calculate_pretrain_approach(self, n_companies=20):
        """Calculate cost with pre-training approach"""
        # One-time pre-training cost
        pretrain_cost = self.cost_per_model
        pretrain_time = self.training_time
        
        # Fine-tuning cost per company (10% of full training)
        finetune_cost_per = self.cost_per_model * 0.1
        finetune_time_per = self.training_time * 0.1
        
        total_cost = pretrain_cost + (n_companies * finetune_cost_per)
        total_time = pretrain_time + (n_companies * finetune_time_per)
        total_gpu_cost = total_time * self.gpu_cost_per_hour
        
        # Carbon footprint
        total_carbon = total_time * self.kwh_per_gpu_hour * self.carbon_per_kwh
        
        return {
            'total_cost': total_cost,
            'duplicate_cost': 0,  # No duplication!
            'total_time': total_time,
            'gpu_cost': total_gpu_cost,
            'carbon_kg': total_carbon
        }
    
    def compare_approaches(self, n_companies=20):
        """Compare traditional vs pre-training approaches"""
        traditional = self.calculate_traditional_approach(n_companies)
        pretrain = self.calculate_pretrain_approach(n_companies)
        
        savings = {
            'cost_saved': traditional['total_cost'] - pretrain['total_cost'],
            'time_saved': traditional['total_time'] - pretrain['total_time'],
            'carbon_saved': traditional['carbon_kg'] - pretrain['carbon_kg'],
            'efficiency_gain': traditional['total_cost'] / pretrain['total_cost']
        }
        
        return traditional, pretrain, savings

# Create calculator and run comparison
calculator = ParadigmShiftCalculator()
traditional, pretrain, savings = calculator.compare_approaches(20)

print("="*70)
print("THE $10 MILLION PROBLEM: 20 Companies Training NLP Models")
print("="*70)
print("\n🔴 TRADITIONAL APPROACH (Everyone trains from scratch):")
print(f"   Total Cost: ${traditional['total_cost']:,.0f}")
print(f"   Wasted on Duplication: ${traditional['duplicate_cost']:,.0f} (90%!)")
print(f"   Total GPU Hours: {traditional['total_time']:,.0f}")
print(f"   Carbon Footprint: {traditional['carbon_kg']:,.0f} kg CO2")
print(f"   Equivalent to: {traditional['carbon_kg']/5000:.0f} cars driven for a year")

print("\n🟢 PRE-TRAINING APPROACH (Train once, fine-tune many):")
print(f"   Total Cost: ${pretrain['total_cost']:,.0f}")
print(f"   Wasted on Duplication: $0")
print(f"   Total GPU Hours: {pretrain['total_time']:,.0f}")
print(f"   Carbon Footprint: {pretrain['carbon_kg']:,.0f} kg CO2")

print("\n💰 SAVINGS WITH PRE-TRAINING:")
print(f"   Money Saved: ${savings['cost_saved']:,.0f}")
print(f"   Time Saved: {savings['time_saved']:,.0f} GPU hours")
print(f"   Carbon Saved: {savings['carbon_saved']:,.0f} kg CO2")
print(f"   Efficiency Gain: {savings['efficiency_gain']:.1f}x")
print("="*70)

In [ ]:
# Interactive: Visualize the Paradigm Shift
def visualize_paradigm_shift(n_companies_list=[5, 10, 20, 50]):
    """Create interactive visualization of cost savings"""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('The Paradigm Shift: From Wasteful Duplication to Efficient Sharing', 
                 fontsize=16, fontweight='bold')
    
    traditional_costs = []
    pretrain_costs = []
    
    for n in n_companies_list:
        trad, pre, _ = calculator.compare_approaches(n)
        traditional_costs.append(trad['total_cost'] / 1_000_000)  # Convert to millions
        pretrain_costs.append(pre['total_cost'] / 1_000_000)
    
    # Cost comparison
    ax1 = axes[0, 0]
    x = np.arange(len(n_companies_list))
    width = 0.35
    ax1.bar(x - width/2, traditional_costs, width, label='Traditional', color='#e74c3c', alpha=0.8)
    ax1.bar(x + width/2, pretrain_costs, width, label='Pre-training', color='#2ecc71', alpha=0.8)
    ax1.set_xlabel('Number of Companies')
    ax1.set_xticks(x)
    ax1.set_xticklabels(n_companies_list)
    ax1.set_ylabel('Total Cost (Millions USD)')
    ax1.set_title('Total Cost Comparison')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Savings over companies
    ax2 = axes[0, 1]
    savings = [t - p for t, p in zip(traditional_costs, pretrain_costs)]
    ax2.plot(n_companies_list, savings, 'o-', color='#3498db', linewidth=2, markersize=8)
    ax2.fill_between(n_companies_list, 0, savings, alpha=0.3, color='#3498db')
    ax2.set_xlabel('Number of Companies')
    ax2.set_ylabel('Money Saved (Millions USD)')
    ax2.set_title('Cumulative Savings with Pre-training')
    ax2.grid(True, alpha=0.3)
    
    # Carbon footprint
    ax3 = axes[1, 0]
    carbon_trad = []
    carbon_pre = []
    for n in n_companies_list:
        trad, pre, _ = calculator.compare_approaches(n)
        carbon_trad.append(trad['carbon_kg'] / 1000)  # Convert to tons
        carbon_pre.append(pre['carbon_kg'] / 1000)
    
    ax3.bar(x - width/2, carbon_trad, width, label='Traditional', color='#95a5a6', alpha=0.8)
    ax3.bar(x + width/2, carbon_pre, width, label='Pre-training', color='#27ae60', alpha=0.8)
    ax3.set_xlabel('Number of Companies')
    ax3.set_xticks(x)
    ax3.set_xticklabels(n_companies_list)
    ax3.set_ylabel('Carbon Emissions (Tons CO2)')
    ax3.set_title('Environmental Impact')
    ax3.legend()
    ax3.grid(axis='y', alpha=0.3)
    
    # Efficiency gain
    ax4 = axes[1, 1]
    efficiency = [t / p for t, p in zip(traditional_costs, pretrain_costs)]
    colors_eff = ['#f39c12', '#e67e22', '#d35400', '#c0392b']
    bars = ax4.bar(n_companies_list, efficiency, color=colors_eff, alpha=0.8, edgecolor='black', linewidth=2)
    ax4.set_xlabel('Number of Companies')
    ax4.set_ylabel('Efficiency Gain (x times)')
    ax4.set_title('Efficiency Multiplier')
    ax4.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, eff in zip(bars, efficiency):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{eff:.1f}x', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Run the visualization
visualize_paradigm_shift()

## NEW: The Paradigm Shift - Interactive Demonstration

Let's start by understanding the revolutionary impact of pre-training through an interactive visualization.

## Part 2: Understanding BERT's Masked Language Modeling

Let's start by exploring how BERT was pre-trained using masked language modeling (MLM).

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
mlm_model = BertForMaskedLM.from_pretrained(model_name)
mlm_model.eval()  # Set to evaluation mode

print(f"Model loaded: {model_name}")
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Model parameters: {sum(p.numel() for p in mlm_model.parameters()):,}")

In [ ]:
def predict_masked_word(text, masked_index=None):
    """
    Predict masked word in a sentence using BERT
    """
    # Replace word with [MASK] if index provided
    words = text.split()
    if masked_index is not None:
        words[masked_index] = '[MASK]'
    masked_text = ' '.join(words)
    
    # Tokenize
    inputs = tokenizer(masked_text, return_tensors='pt')
    
    # Get predictions
    with torch.no_grad():
        outputs = mlm_model(**inputs)
        predictions = outputs.logits
    
    # Find masked token position
    mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
    
    # Get top 5 predictions
    masked_token_logits = predictions[0, mask_token_index, :]
    top_5_tokens = torch.topk(masked_token_logits, 5, dim=1).indices[0].tolist()
    
    print(f"Original text: {text}")
    print(f"Masked text: {masked_text}")
    print("\nTop 5 predictions:")
    for i, token in enumerate(top_5_tokens, 1):
        word = tokenizer.decode([token])
        prob = torch.softmax(masked_token_logits, dim=1)[0, token].item()
        print(f"{i}. {word:15} (probability: {prob:.2%})")
    
    return top_5_tokens

# Test the function
predict_masked_word("The cat sat on the mat", masked_index=1)

In [ ]:
# Interactive MLM examples
examples = [
    "The weather today is [MASK]",
    "I love to eat [MASK] for breakfast",
    "The [MASK] is the capital of France",
    "She is a [MASK] student at the university",
    "The movie was [MASK] than I expected"
]

print("BERT's Masked Language Model Predictions:\n")
print("="*50)

for example in examples:
    inputs = tokenizer(example, return_tensors='pt')
    mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
    
    with torch.no_grad():
        outputs = mlm_model(**inputs)
        predictions = outputs.logits
    
    masked_token_logits = predictions[0, mask_token_index, :]
    top_token = torch.topk(masked_token_logits, 1, dim=1).indices[0].item()
    predicted_word = tokenizer.decode([top_token])
    
    print(f"Input:  {example}")
    print(f"Prediction: {example.replace('[MASK]', f'**{predicted_word}**')}")
    print("-"*50)

## Part 3: Fine-tuning BERT for Sentiment Analysis

Now let's fine-tune BERT for a specific task: sentiment analysis on movie reviews.

In [ ]:
# Load IMDB dataset (subset for faster training)
print("Loading IMDB dataset...")
dataset = load_dataset('imdb')

# Use smaller subset for demo
train_size = 1000
test_size = 200

train_dataset = dataset['train'].select(range(train_size))
test_dataset = dataset['test'].select(range(test_size))

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"\nExample review:")
print(f"Text: {train_dataset[0]['text'][:200]}...")
print(f"Label: {'Positive' if train_dataset[0]['label'] == 1 else 'Negative'}")

In [ ]:
# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

print("Tokenizing datasets...")
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

print("Tokenization complete!")

In [ ]:
# Load pre-trained BERT for sequence classification
model_ft = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    output_attentions=True,  # We'll use this for visualization
    output_hidden_states=True
)

print(f"Model loaded for fine-tuning")
print(f"Number of trainable parameters: {sum(p.numel() for p in model_ft.parameters() if p.requires_grad):,}")

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Define metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Create trainer
trainer = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

print("Trainer initialized. Ready to fine-tune!")

In [ ]:
# Fine-tune the model
print("Starting fine-tuning...")
print("This will take a few minutes...\n")

# Store initial time
import time
start_time = time.time()

# Train
train_result = trainer.train()

# Calculate training time
training_time = time.time() - start_time
print(f"\nFine-tuning complete!")
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Final training loss: {train_result.training_loss:.4f}")

## NEW: Environmental Impact Calculator

Let's calculate the environmental impact of training different model sizes.

In [ ]:
# Evaluate the model
eval_result = trainer.evaluate()
print(f"Test Accuracy: {eval_result['eval_accuracy']:.2%}")
print(f"Test Loss: {eval_result['eval_loss']:.4f}")

## Part 4: Comparing with Training from Scratch

Let's create a BERT model from scratch (randomly initialized) and see how it performs.

In [ ]:
# Create BERT from scratch (random initialization)
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    num_labels=2
)

model_scratch = BertForSequenceClassification(config)
print(f"Created BERT from scratch with random weights")
print(f"Number of parameters: {sum(p.numel() for p in model_scratch.parameters()):,}")

In [ ]:
# Train from scratch (same settings)
trainer_scratch = Trainer(
    model=model_scratch,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

print("Training from scratch...")
print("This will take a few minutes...\n")

start_time = time.time()
train_result_scratch = trainer_scratch.train()
scratch_time = time.time() - start_time

print(f"\nTraining from scratch complete!")
print(f"Training time: {scratch_time/60:.2f} minutes")
print(f"Final training loss: {train_result_scratch.training_loss:.4f}")

In [ ]:
# Compare results
eval_scratch = trainer_scratch.evaluate()

# Create comparison
comparison_df = pd.DataFrame({
    'Metric': ['Test Accuracy', 'Test Loss', 'Training Time (min)'],
    'Pre-trained + Fine-tuned': [
        f"{eval_result['eval_accuracy']:.2%}",
        f"{eval_result['eval_loss']:.4f}",
        f"{training_time/60:.2f}"
    ],
    'Trained from Scratch': [
        f"{eval_scratch['eval_accuracy']:.2%}",
        f"{eval_scratch['eval_loss']:.4f}",
        f"{scratch_time/60:.2f}"
    ]
})

print("\n" + "="*50)
print("COMPARISON: Pre-trained vs From Scratch")
print("="*50)
print(comparison_df.to_string(index=False))
print("="*50)
print(f"\nImprovement from pre-training: {(eval_result['eval_accuracy'] - eval_scratch['eval_accuracy'])*100:.1f} percentage points!")

## Part 5: Using the Fine-tuned Model

Let's test our fine-tuned model on some new movie reviews.

In [ ]:
# Create a sentiment analysis pipeline
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model_ft,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# Test reviews
test_reviews = [
    "This movie was absolutely fantastic! Best film I've seen all year.",
    "Terrible waste of time. The plot made no sense and the acting was awful.",
    "Not bad, but not great either. It was just okay.",
    "A masterpiece of cinema! Every scene was perfectly crafted.",
    "I fell asleep halfway through. Boring and predictable.",
    "Surprisingly good! I wasn't expecting much but was pleasantly surprised."
]

print("Sentiment Analysis Results:")
print("="*60)

for review in test_reviews:
    result = sentiment_pipeline(review)[0]
    label = "Positive 😊" if result['label'] == 'LABEL_1' else "Negative 😞"
    confidence = result['score']
    
    print(f"Review: {review[:50]}...")
    print(f"Sentiment: {label} (Confidence: {confidence:.2%})")
    print("-"*60)

## NEW: LoRA - Low-Rank Adaptation for Efficient Fine-tuning

Let's implement LoRA to see how we can fine-tune large models efficiently.

## Part 6: Visualizing Attention Patterns

Let's visualize what BERT is "paying attention to" when making predictions.

In [ ]:
def get_attention_weights(text, model, tokenizer):
    """
    Extract attention weights from BERT
    """
    # Tokenize
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128)
    
    # Get model outputs with attention
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract attention weights (last layer, first head)
    attention = outputs.attentions[-1][0, 0, :, :].numpy()
    
    # Get tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    return attention, tokens

# Example sentence
example_text = "This movie was absolutely terrible!"
attention, tokens = get_attention_weights(example_text, model_ft, tokenizer)

# Visualize attention matrix
plt.figure(figsize=(10, 8))
sns.heatmap(attention[:len(tokens), :len(tokens)], 
            xticklabels=tokens,
            yticklabels=tokens,
            cmap='Blues',
            cbar_kws={'label': 'Attention Weight'})
plt.title(f'BERT Attention Patterns\nText: "{example_text}"', fontsize=14)
plt.xlabel('Keys (Tokens)', fontsize=12)
plt.ylabel('Queries (Tokens)', fontsize=12)
plt.tight_layout()
plt.show()

# Analyze which words get most attention
avg_attention = attention.mean(axis=0)[:len(tokens)]
top_attention_idx = avg_attention.argsort()[-5:][::-1]

print("\nWords receiving most attention (averaged):")
for idx in top_attention_idx:
    if tokens[idx] not in ['[CLS]', '[SEP]', '[PAD]']:
        print(f"- {tokens[idx]}: {avg_attention[idx]:.3f}")

## NEW: BERT vs GPT - Interactive Comparison

Let's interactively explore the differences between bidirectional (BERT) and autoregressive (GPT) models.

## Part 7: Key Insights and Takeaways

Let's summarize what we've learned about pre-trained models.

## Enhanced Conclusion - The Paradigm Shift Complete Picture

Through this enhanced notebook, we've explored:

### 🔄 The Paradigm Shift
- **$10 Million Problem**: How pre-training saves millions in compute costs
- **9.1x Efficiency Gain**: Dramatic reduction in resources needed
- **Environmental Impact**: Orders of magnitude reduction in carbon footprint

### 🛠️ Practical Techniques
- **Fine-tuning**: Simple yet powerful transfer learning
- **LoRA**: 100x parameter reduction while maintaining performance
- **Attention Analysis**: Understanding what models "see"

### 🔬 Key Comparisons
- **Pre-trained vs Scratch**: 15-20% accuracy improvement
- **BERT vs GPT**: Different tools for different tasks
- **Environmental Costs**: From tons to kilograms of CO2

### 💡 Main Takeaways
1. **Never train from scratch** - Stand on the shoulders of giants
2. **Choose the right model** - BERT for understanding, GPT for generation
3. **Use efficient methods** - LoRA, quantization, pruning
4. **Consider the environment** - Every parameter counts

### 🚀 Your Next Steps
1. Try fine-tuning on your own dataset
2. Experiment with different model sizes
3. Implement LoRA for your specific use case
4. Explore prompt engineering for even better results

**Remember**: The paradigm shift isn't just about better models - it's about democratizing AI by making powerful models accessible to everyone!

In [ ]:
# Create visualization comparing training approaches
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
ax1 = axes[0]
categories = ['Pre-trained\n+ Fine-tuned', 'Trained from\nScratch']
accuracies = [eval_result['eval_accuracy'], eval_scratch['eval_accuracy']]
colors = ['#2ecc71', '#e74c3c']

bars1 = ax1.bar(categories, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
ax1.set_ylabel('Test Accuracy', fontsize=12)
ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 1)
ax1.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, acc in zip(bars1, accuracies):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{acc:.1%}', ha='center', va='bottom', fontsize=12, fontweight='bold')

# Training efficiency
ax2 = axes[1]
times = [training_time/60, scratch_time/60]
bars2 = ax2.bar(categories, times, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
ax2.set_ylabel('Training Time (minutes)', fontsize=12)
ax2.set_title('Training Efficiency', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for bar, time in zip(bars2, times):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{time:.1f} min', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.suptitle('The Power of Pre-training: Better Performance, Faster Training', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*60)
print("SUMMARY: The Benefits of Pre-training")
print("="*60)
print(f"✅ Accuracy Improvement: {(accuracies[0] - accuracies[1])*100:.1f} percentage points")
print(f"✅ Training Speedup: {times[1]/times[0]:.1f}x faster to reach better performance")
print(f"✅ Data Efficiency: Same amount of data, much better results")
print(f"✅ Robustness: Pre-trained models generalize better")
print("="*60)

## Exercises and Challenges

Now it's your turn to experiment! Try these exercises:

### Exercise 1: Different Masking Strategies
Modify the MLM example to mask multiple words and see how BERT handles it.

### Exercise 2: Fine-tune for Different Tasks
Try fine-tuning BERT for:
- Text classification on a different dataset (e.g., news categories)
- Named Entity Recognition
- Question Answering

### Exercise 3: Compare Model Sizes
Try using different BERT variants:
- `bert-tiny` (very small, fast)
- `bert-base` (what we used)
- `bert-large` (bigger, more accurate)

### Exercise 4: Attention Analysis
Analyze attention patterns for different types of sentences:
- Questions
- Negations
- Complex sentences with multiple clauses

In [ ]:
# Space for your experiments
# Try Exercise 1: Multiple masked words
text_with_masks = "The [MASK] cat [MASK] on the [MASK]."
print("Your turn to experiment!")
print(f"Try predicting: {text_with_masks}")

# Your code here...

## Conclusion

In this notebook, we've:
1. ✅ Explored BERT's masked language modeling
2. ✅ Fine-tuned BERT for sentiment analysis
3. ✅ Compared with training from scratch
4. ✅ Visualized attention patterns
5. ✅ Demonstrated the power of pre-training

### Key Takeaways:
- **Pre-training provides a massive head start** - better performance with less data and time
- **Fine-tuning is simple** - just a few lines of code with Hugging Face Transformers
- **Attention patterns are interpretable** - we can see what the model focuses on
- **Transfer learning works** - knowledge from pre-training transfers to new tasks

### Next Steps:
- Try fine-tuning for your own task
- Explore other pre-trained models (GPT, T5, RoBERTa)
- Learn about prompt engineering for even better results
- Experiment with larger models if you have GPU access

**Remember**: You don't need to train from scratch - stand on the shoulders of giants!

In [ ]:
class EnvironmentalImpactCalculator:
    """Calculate carbon footprint of different model training approaches"""
    
    def __init__(self):
        # Model sizes and training requirements
        self.models = {
            'BERT-Tiny': {'params': 4.4e6, 'gpu_hours': 24, 'flops': 1e18},
            'BERT-Mini': {'params': 11.3e6, 'gpu_hours': 72, 'flops': 1e19},
            'BERT-Small': {'params': 29.1e6, 'gpu_hours': 168, 'flops': 5e19},
            'BERT-Base': {'params': 110e6, 'gpu_hours': 336, 'flops': 2e20},
            'BERT-Large': {'params': 340e6, 'gpu_hours': 1008, 'flops': 7e20},
            'GPT-2': {'params': 1.5e9, 'gpu_hours': 2016, 'flops': 3e21},
            'GPT-3': {'params': 175e9, 'gpu_hours': 355000, 'flops': 3.14e23},
        }
        
        # Environmental constants
        self.kwh_per_gpu_hour = 0.4  # kilowatt hours
        self.co2_per_kwh = {
            'US Average': 0.42,
            'Coal': 0.82,
            'Natural Gas': 0.49,
            'Nuclear': 0.012,
            'Solar': 0.048,
            'Wind': 0.011,
            'Hydro': 0.024
        }
        
        # Real-world comparisons
        self.car_km_per_year = 15000
        self.co2_per_car_km = 0.12  # kg CO2 per km
        self.tree_co2_per_year = 21  # kg CO2 absorbed per tree per year
        self.flight_co2_per_km = 0.115  # kg CO2 per passenger km
    
    def calculate_carbon_footprint(self, model_name, energy_source='US Average', 
                                  num_experiments=1, include_hyperparameter_search=False):
        """Calculate carbon footprint for training a model"""
        
        if model_name not in self.models:
            return None
        
        model = self.models[model_name]
        gpu_hours = model['gpu_hours']
        
        # Add hyperparameter search overhead (typically 3-10x)
        if include_hyperparameter_search:
            gpu_hours *= 5
        
        # Multiple experiments
        gpu_hours *= num_experiments
        
        # Calculate energy and CO2
        energy_kwh = gpu_hours * self.kwh_per_gpu_hour
        co2_kg = energy_kwh * self.co2_per_kwh[energy_source]
        
        # Calculate comparisons
        car_years = co2_kg / (self.car_km_per_year * self.co2_per_car_km / 1000)
        trees_needed = co2_kg / self.tree_co2_per_year
        flight_km = co2_kg / self.flight_co2_per_km
        
        return {
            'model': model_name,
            'gpu_hours': gpu_hours,
            'energy_kwh': energy_kwh,
            'co2_kg': co2_kg,
            'co2_tons': co2_kg / 1000,
            'car_years': car_years,
            'trees_needed': trees_needed,
            'flight_km': flight_km,
            'energy_source': energy_source
        }
    
    def compare_models(self, model_names, energy_source='US Average'):
        """Compare carbon footprint of multiple models"""
        results = []
        for model in model_names:
            result = self.calculate_carbon_footprint(model, energy_source)
            if result:
                results.append(result)
        return results
    
    def visualize_comparison(self, model_names=None):
        """Create visualization comparing model carbon footprints"""
        if model_names is None:
            model_names = ['BERT-Tiny', 'BERT-Base', 'BERT-Large', 'GPT-2']
        
        results = self.compare_models(model_names)
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Environmental Impact of Model Training', fontsize=16, fontweight='bold')
        
        # CO2 emissions
        ax1 = axes[0, 0]
        models = [r['model'] for r in results]
        co2_tons = [r['co2_tons'] for r in results]
        colors = plt.cm.Reds(np.linspace(0.4, 0.9, len(models)))
        bars1 = ax1.bar(models, co2_tons, color=colors, edgecolor='black', linewidth=2)
        ax1.set_ylabel('CO2 Emissions (Tons)')
        ax1.set_title('Carbon Footprint by Model Size')
        ax1.set_yscale('log')
        ax1.grid(axis='y', alpha=0.3)
        
        # Add value labels
        for bar, val in zip(bars1, co2_tons):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height * 1.1,
                    f'{val:.2f}t', ha='center', va='bottom', fontweight='bold')
        
        # Energy consumption
        ax2 = axes[0, 1]
        energy = [r['energy_kwh'] for r in results]
        bars2 = ax2.bar(models, energy, color=colors, edgecolor='black', linewidth=2)
        ax2.set_ylabel('Energy (kWh)')
        ax2.set_title('Energy Consumption')
        ax2.set_yscale('log')
        ax2.grid(axis='y', alpha=0.3)
        
        # Car year equivalents
        ax3 = axes[1, 0]
        car_years = [r['car_years'] for r in results]
        bars3 = ax3.bar(models, car_years, color=colors, edgecolor='black', linewidth=2)
        ax3.set_ylabel('Car Years Equivalent')
        ax3.set_title('Equivalent to Driving a Car For...')
        ax3.grid(axis='y', alpha=0.3)
        
        # Trees needed to offset
        ax4 = axes[1, 1]
        trees = [r['trees_needed'] for r in results]
        bars4 = ax4.bar(models, trees, color=colors, edgecolor='black', linewidth=2)
        ax4.set_ylabel('Number of Trees')
        ax4.set_title('Trees Needed to Offset (1 Year)')
        ax4.set_yscale('log')
        ax4.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        return results

# Create calculator and run comparison
env_calculator = EnvironmentalImpactCalculator()

print("="*70)
print("ENVIRONMENTAL IMPACT ANALYSIS")
print("="*70)

# Calculate for BERT-Base with different energy sources
print("\nBERT-Base Training - Impact by Energy Source:")
print("-"*50)

for source in ['Coal', 'Natural Gas', 'US Average', 'Nuclear', 'Solar']:
    result = env_calculator.calculate_carbon_footprint('BERT-Base', source)
    print(f"{source:15} -> {result['co2_kg']:.1f} kg CO2 ({result['co2_tons']:.2f} tons)")

print("\n" + "="*70)
print("COMPARISON: Different Model Sizes")
print("="*70)

# Visualize comparison
env_calculator.visualize_comparison()

In [ ]:
class LoRALayer(nn.Module):
    """
    Implements Low-Rank Adaptation (LoRA) for efficient fine-tuning
    Instead of updating all weights W, we learn low-rank matrices A and B
    such that W' = W + BA where B is d x r and A is r x k (r << min(d,k))
    """
    
    def __init__(self, original_layer, rank=4, alpha=1):
        super().__init__()
        self.original_layer = original_layer
        self.rank = rank
        self.alpha = alpha
        
        # Get dimensions
        if hasattr(original_layer, 'in_features'):
            # Linear layer
            d_in = original_layer.in_features
            d_out = original_layer.out_features
            self.is_linear = True
        else:
            raise ValueError("LoRA only supports Linear layers in this implementation")
        
        # Create low-rank matrices
        self.lora_A = nn.Parameter(torch.randn(rank, d_in) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(d_out, rank))
        
        # Freeze original weights
        for param in self.original_layer.parameters():
            param.requires_grad = False
        
        # Scaling factor
        self.scaling = self.alpha / self.rank
    
    def forward(self, x):
        # Original forward pass
        result = self.original_layer(x)
        
        # Add LoRA adaptation
        if self.is_linear:
            # x @ A.T @ B.T * scaling
            lora_out = x @ self.lora_A.T @ self.lora_B.T * self.scaling
            result = result + lora_out
        
        return result
    
    def merge_weights(self):
        """Merge LoRA weights into original layer for inference"""
        if self.is_linear:
            delta_w = (self.lora_B @ self.lora_A) * self.scaling
            self.original_layer.weight.data += delta_w

# Demonstrate LoRA parameter efficiency
def calculate_lora_parameters(model_params, lora_rank, num_adapted_layers):
    """Calculate parameter reduction with LoRA"""
    
    # Assume adapting attention layers (typical scenario)
    # BERT-Base has 768 hidden dim
    hidden_dim = 768
    
    # Original parameters to update (full fine-tuning)
    original_trainable = model_params
    
    # LoRA parameters per adapted layer
    # For Q, K, V projections: 3 * (rank * hidden_dim * 2)
    lora_params_per_layer = 3 * (lora_rank * hidden_dim * 2)
    total_lora_params = lora_params_per_layer * num_adapted_layers
    
    # Calculate reduction
    reduction = original_trainable / total_lora_params
    percentage = (total_lora_params / original_trainable) * 100
    
    return {
        'original_params': original_trainable,
        'lora_params': total_lora_params,
        'reduction_factor': reduction,
        'percentage_of_original': percentage
    }

# Visualize LoRA efficiency
def visualize_lora_efficiency():
    """Create visualization of LoRA parameter efficiency"""
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('LoRA: Efficient Fine-tuning Through Low-Rank Adaptation', 
                 fontsize=14, fontweight='bold')
    
    # Different model sizes
    models = {
        'BERT-Base': 110e6,
        'BERT-Large': 340e6,
        'GPT-2': 1.5e9,
        'GPT-3-13B': 13e9
    }
    
    # LoRA configurations
    ranks = [1, 4, 8, 16, 32]
    
    # Parameter reduction for different ranks
    ax1 = axes[0]
    for model_name, params in models.items():
        reductions = []
        for rank in ranks:
            result = calculate_lora_parameters(params, rank, 12)  # 12 transformer layers
            reductions.append(result['percentage_of_original'])
        ax1.plot(ranks, reductions, 'o-', label=model_name, linewidth=2, markersize=6)
    
    ax1.set_xlabel('LoRA Rank (r)')
    ax1.set_ylabel('Trainable Parameters (%)')
    ax1.set_title('Parameter Efficiency vs LoRA Rank')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    # Memory savings
    ax2 = axes[1]
    model_names = list(models.keys())
    memory_full = [m/1e9 * 4 for m in models.values()]  # 4 bytes per param
    memory_lora = []
    
    for params in models.values():
        result = calculate_lora_parameters(params, 8, 12)  # rank=8
        memory_lora.append(result['lora_params']/1e9 * 4)
    
    x = np.arange(len(model_names))
    width = 0.35
    ax2.bar(x - width/2, memory_full, width, label='Full Fine-tuning', color='#e74c3c', alpha=0.8)
    ax2.bar(x + width/2, memory_lora, width, label='LoRA (r=8)', color='#2ecc71', alpha=0.8)
    ax2.set_xlabel('Model')
    ax2.set_xticks(x)
    ax2.set_xticklabels(model_names, rotation=45, ha='right')
    ax2.set_ylabel('Memory (GB)')
    ax2.set_title('Memory Requirements')
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    # Training speedup
    ax3 = axes[2]
    speedup_factors = []
    for params in models.values():
        result = calculate_lora_parameters(params, 8, 12)
        # Approximate speedup based on parameter reduction
        speedup = min(result['reduction_factor'] * 0.7, 10)  # Cap at 10x
        speedup_factors.append(speedup)
    
    bars = ax3.bar(model_names, speedup_factors, color='#3498db', alpha=0.8, edgecolor='black', linewidth=2)
    ax3.set_xlabel('Model')
    ax3.set_ylabel('Training Speedup (x)')
    ax3.set_title('Training Speed Improvement')
    ax3.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, val in zip(bars, speedup_factors):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{val:.1f}x', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Example: Apply LoRA to a layer
print("LoRA DEMONSTRATION")
print("="*60)

# Create a dummy linear layer (like in BERT)
original_linear = nn.Linear(768, 768)
print(f"Original layer parameters: {sum(p.numel() for p in original_linear.parameters()):,}")

# Wrap with LoRA
lora_layer = LoRALayer(original_linear, rank=8, alpha=16)
trainable_params = sum(p.numel() for p in lora_layer.parameters() if p.requires_grad)
print(f"LoRA trainable parameters: {trainable_params:,}")
print(f"Parameter reduction: {sum(p.numel() for p in original_linear.parameters()) / trainable_params:.1f}x")

print("\n" + "="*60)
print("EFFICIENCY COMPARISON")
print("="*60)

# Calculate for BERT-Base
bert_result = calculate_lora_parameters(110e6, 8, 12)
print(f"BERT-Base Full Fine-tuning: {bert_result['original_params']/1e6:.1f}M parameters")
print(f"BERT-Base with LoRA (r=8):  {bert_result['lora_params']/1e6:.2f}M parameters")
print(f"Reduction factor: {bert_result['reduction_factor']:.1f}x")
print(f"Only training {bert_result['percentage_of_original']:.2f}% of original parameters!")

# Visualize
visualize_lora_efficiency()

In [ ]:
class BERTvsGPTComparison:
    """Interactive comparison of BERT and GPT architectures"""
    
    def __init__(self):
        self.bert_features = {
            'architecture': 'Bidirectional',
            'pretraining': 'Masked Language Modeling (MLM)',
            'objective': 'Predict masked tokens',
            'context': 'Sees both left and right context',
            'best_for': ['Classification', 'Token labeling', 'Question answering'],
            'limitations': ['Cannot generate text naturally', 'Requires task-specific heads'],
            'example_models': ['BERT', 'RoBERTa', 'ELECTRA', 'DeBERTa']
        }
        
        self.gpt_features = {
            'architecture': 'Autoregressive',
            'pretraining': 'Next Token Prediction',
            'objective': 'Predict next token',
            'context': 'Sees only left context',
            'best_for': ['Text generation', 'Completion', 'Few-shot learning'],
            'limitations': ['No bidirectional context', 'May need more data for understanding'],
            'example_models': ['GPT-2', 'GPT-3', 'GPT-4', 'GPT-J']
        }
    
    def demonstrate_masking_difference(self):
        """Show the difference in how BERT and GPT process text"""
        
        text = "The cat sat on the mat"
        
        print("="*70)
        print("MASKING STRATEGY COMPARISON")
        print("="*70)
        print(f"Original text: '{text}'")
        print("-"*70)
        
        # BERT-style masking
        print("\nBERT (Masked Language Modeling):")
        print("  Training example: 'The [MASK] sat on the mat'")
        print("  Task: Predict what [MASK] is")
        print("  Context used: 'The' (left) + 'sat on the mat' (right)")
        print("  -> Can see BOTH directions!")
        
        # GPT-style prediction
        print("\nGPT (Autoregressive):")
        print("  Training examples (processed sequentially):")
        print("    Step 1: 'The' -> predict 'cat'")
        print("    Step 2: 'The cat' -> predict 'sat'")
        print("    Step 3: 'The cat sat' -> predict 'on'")
        print("  -> Can only see LEFT context!")
        
        print("="*70)
    
    def visualize_attention_patterns(self):
        """Visualize how BERT and GPT attend to tokens differently"""
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle('Attention Pattern Comparison: BERT vs GPT', 
                     fontsize=14, fontweight='bold')
        
        tokens = ['The', 'cat', 'sat', 'on', 'the', 'mat']
        n_tokens = len(tokens)
        
        # BERT attention (bidirectional)
        ax1 = axes[0]
        bert_attention = np.ones((n_tokens, n_tokens))  # Can attend to all
        im1 = ax1.imshow(bert_attention, cmap='Blues', vmin=0, vmax=1)
        ax1.set_xticks(range(n_tokens))
        ax1.set_yticks(range(n_tokens))
        ax1.set_xticklabels(tokens, rotation=45)
        ax1.set_yticklabels(tokens)
        ax1.set_title('BERT: Bidirectional Attention')
        ax1.set_xlabel('Keys')
        ax1.set_ylabel('Queries')
        
        # Add grid
        for i in range(n_tokens + 1):
            ax1.axhline(i - 0.5, color='gray', linewidth=0.5)
            ax1.axvline(i - 0.5, color='gray', linewidth=0.5)
        
        # GPT attention (causal/autoregressive)
        ax2 = axes[1]
        gpt_attention = np.tril(np.ones((n_tokens, n_tokens)))  # Lower triangular
        im2 = ax2.imshow(gpt_attention, cmap='Oranges', vmin=0, vmax=1)
        ax2.set_xticks(range(n_tokens))
        ax2.set_yticks(range(n_tokens))
        ax2.set_xticklabels(tokens, rotation=45)
        ax2.set_yticklabels(tokens)
        ax2.set_title('GPT: Causal (Autoregressive) Attention')
        ax2.set_xlabel('Keys')
        ax2.set_ylabel('Queries')
        
        # Add grid
        for i in range(n_tokens + 1):
            ax2.axhline(i - 0.5, color='gray', linewidth=0.5)
            ax2.axvline(i - 0.5, color='gray', linewidth=0.5)
        
        # Add text annotations
        ax1.text(0.5, -0.15, 'Each token can attend to ALL other tokens',
                transform=ax1.transAxes, ha='center', fontsize=10, style='italic')
        ax2.text(0.5, -0.15, 'Each token can only attend to PREVIOUS tokens',
                transform=ax2.transAxes, ha='center', fontsize=10, style='italic')
        
        plt.tight_layout()
        plt.show()
    
    def compare_use_cases(self):
        """Create comparison chart of use cases"""
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle('BERT vs GPT: Best Use Cases', fontsize=14, fontweight='bold')
        
        # Task performance comparison
        tasks = ['Text\nClassification', 'Named Entity\nRecognition', 
                'Question\nAnswering', 'Text\nGeneration', 
                'Story\nCompletion', 'Few-shot\nLearning']
        
        bert_scores = [95, 92, 90, 30, 40, 50]  # Relative performance
        gpt_scores = [80, 70, 75, 95, 92, 90]
        
        ax1 = axes[0]
        x = np.arange(len(tasks))
        width = 0.35
        
        bars1 = ax1.bar(x - width/2, bert_scores, width, label='BERT', 
                       color='#3498db', alpha=0.8, edgecolor='black', linewidth=1)
        bars2 = ax1.bar(x + width/2, gpt_scores, width, label='GPT', 
                       color='#e67e22', alpha=0.8, edgecolor='black', linewidth=1)
        
        ax1.set_xlabel('Task Type')
        ax1.set_ylabel('Relative Performance')
        ax1.set_title('Task Performance Comparison')
        ax1.set_xticks(x)
        ax1.set_xticklabels(tasks)
        ax1.legend()
        ax1.grid(axis='y', alpha=0.3)
        ax1.set_ylim(0, 100)
        
        # Characteristics radar chart
        ax2 = axes[1]
        categories = ['Understanding', 'Generation', 'Bidirectional\nContext', 
                     'Few-shot', 'Speed', 'Memory\nEfficiency']
        
        bert_values = [90, 30, 100, 40, 70, 80]
        gpt_values = [75, 100, 0, 90, 60, 60]
        
        # Number of variables
        num_vars = len(categories)
        
        # Compute angle for each axis
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        
        # Complete the circle
        bert_values += bert_values[:1]
        gpt_values += gpt_values[:1]
        angles += angles[:1]
        
        # Plot
        ax2 = plt.subplot(122, projection='polar')
        ax2.plot(angles, bert_values, 'o-', linewidth=2, label='BERT', color='#3498db')
        ax2.fill(angles, bert_values, alpha=0.25, color='#3498db')
        ax2.plot(angles, gpt_values, 'o-', linewidth=2, label='GPT', color='#e67e22')
        ax2.fill(angles, gpt_values, alpha=0.25, color='#e67e22')
        
        ax2.set_xticks(angles[:-1])
        ax2.set_xticklabels(categories)
        ax2.set_ylim(0, 100)
        ax2.set_title('Model Characteristics', y=1.1)
        ax2.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()
    
    def interactive_comparison_table(self):
        """Create detailed comparison table"""
        
        comparison_data = {
            'Aspect': [
                'Architecture',
                'Pre-training Task',
                'Context Direction',
                'Best for Understanding',
                'Best for Generation',
                'Parameter Efficiency',
                'Training Data Needs',
                'Inference Speed',
                'Few-shot Capability',
                'Fine-tuning Ease'
            ],
            'BERT': [
                'Encoder-only',
                'Masked LM + NSP',
                'Bidirectional',
                'Excellent',
                'Poor',
                'High',
                'Moderate',
                'Fast',
                'Limited',
                'Easy'
            ],
            'GPT': [
                'Decoder-only',
                'Next Token Prediction',
                'Left-to-right',
                'Good',
                'Excellent',
                'Moderate',
                'High',
                'Moderate',
                'Excellent',
                'Moderate'
            ],
            'Winner': [
                'Depends on task',
                'GPT (simpler)',
                'BERT',
                'BERT',
                'GPT',
                'BERT',
                'BERT',
                'BERT',
                'GPT',
                'BERT'
            ]
        }
        
        df = pd.DataFrame(comparison_data)
        
        # Create styled display
        print("\n" + "="*80)
        print("COMPREHENSIVE COMPARISON: BERT vs GPT")
        print("="*80)
        
        for _, row in df.iterrows():
            winner_indicator = ""
            if row['Winner'] == 'BERT':
                winner_indicator = " <- BERT wins"
            elif row['Winner'] == 'GPT':
                winner_indicator = " <- GPT wins"
            
            print(f"\n{row['Aspect']}:")
            print(f"  BERT: {row['BERT']}")
            print(f"  GPT:  {row['GPT']}")
            if winner_indicator:
                print(f"  {winner_indicator}")
        
        print("\n" + "="*80)
        
        return df

# Create comparison object and run demonstrations
comparison = BERTvsGPTComparison()

# Show masking difference
comparison.demonstrate_masking_difference()

# Visualize attention patterns
print("\nVisualizing attention patterns...")
comparison.visualize_attention_patterns()

# Compare use cases
print("\nComparing use cases...")
comparison.compare_use_cases()

# Show detailed comparison table
print("\nDetailed comparison table:")
comparison_df = comparison.interactive_comparison_table()